In [ ]:
import numpy as np

import nibabel as nib

import skimage

import matplotlib.pyplot as plt
import os
import os.path as op

from tqdm.notebook import tqdm

import interpolation as interp

In [ ]:
path_to_data = "/Users/acionca/data"
derivative_name = "ants-t1N4bfcorr-ss"

#path_to_data = "/home/acionca/Documents/data"
#derivative_name = "ants-t1N4bfcorr-b80-noSkull"

path_to_imgs = op.join(path_to_data,"hcph-template/multivar-v00/derivatives"
                f"/{derivative_name}")

anat_files_list = interp.get_anat_filenames(path_to_imgs, pattern="corrden",
                                            exclude=["in0048", "rerun"])

path_to_mni = op.join(path_to_data, "hcph-template/mni_template/mni_template-res0.8mm.nii.gz")
ref_img = nib.load(path_to_mni)
ref_data = ref_img.get_fdata()

vmax = 255
scaled_ref_data = ref_data * vmax

path_to_normalized = op.join(path_to_data, "hcph-template/multivar-v00/derivatives"
                      f"/{derivative_name}")

norm_files_list = interp.get_anat_filenames(path_to_normalized, exclude=["in0048", "rerun"])

first_img = nib.load(anat_files_list[0])
#first_img = nib.load(norm_files_list[0])
first_data = first_img.get_fdata()

In [ ]:
anat_files_list_red = anat_files_list#[:2]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
#fig, axes = plt.subplots(nrows=len(anat_files_list_red), ncols=1, figsize=(10, 5*len(anat_files_list_red)))

percentile_to_show = 99
img_percentile = np.percentile(first_data.flatten(), percentile_to_show, method="nearest")+20
nbins=(first_data.max()/2).astype(int)

colors = plt.get_cmap("Spectral", len(anat_files_list_red)+2)

plt.hist(first_data.flatten(), bins=nbins, range=(10, img_percentile), color="tab:blue")

for i, file in enumerate(anat_files_list_red):
    image_data = nib.load(file).get_fdata()

    #plt.hist(image_data.flatten(), bins=nbins, range=(10, img_percentile), color="tab:blue")
    plt.hist(image_data.flatten(), bins=nbins, histtype="step", range=(10, img_percentile), alpha=.6,
             color=colors(i), linewidth=2, label=op.basename(file))
    
fig.legend(loc="upper right", bbox_to_anchor=(1.2, 0.9))

In [ ]:
#path_to_data = op.join(path_to_data, "hcph-template/multivar-v00/derivatives", derivative_name)

anat_files_list = interp.get_anat_filenames(path_to_imgs, pattern="corrden", modality_filter=["T2w"],
                                            exclude=["in0048", "rerun"])

first_img = nib.load(anat_files_list[0])
first_data = first_img.get_fdata()

#saveloc = "/home/acionca/Documents/data/hcph-template/multivar-v00/derivatives/ants-t1N4bfcorr-b80/histomatch"
saveloc = path_to_imgs+"-histomatch"
os.makedirs(saveloc, exist_ok=True)

matched = []
for i, file in enumerate(tqdm(anat_files_list)):
    if i > 20:
        savename = op.basename(file).replace("N4corrden", "N4corrdenhist")
        image = nib.load(file)
        matched_data = skimage.exposure.match_histograms(image.get_fdata(), first_data)
        matched.append(matched_data)

        matched_img = nib.Nifti1Image(matched_data, affine=image.affine, dtype="uint8")
        matched_img.to_filename(op.join(saveloc, savename))
    #matched.append(matched_data)
#
    #axes[i, 0].set_title(f"{op.basename(file)}")
#
    ##img_percentile = np.percentile(image_data.flatten(), 99, method="nearest")+20
    #img_percentile = 256
#
    #nbins=(image_data.max()/2).astype(int)
#
    #axes[i, 0].hist(image_data.flatten(), bins=nbins, range=(0, img_percentile), color="tab:blue")
    #axes[i, 0].hist(matched_data.flatten(), bins=nbins, range=(0, img_percentile), color="tab:orange", alpha=.6)
    #axes[i, 0].axis([0, 256, 0, 60000])
    #
    #axes[i, 1].imshow(image_data[..., 100], cmap="binary_r", vmin=0, vmax=256)
    #axes[i, 2].imshow(matched_data[..., 100], cmap="binary_r", vmin=0, vmax=256)

In [ ]:
fig, axes = plt.subplots(nrows=len(anat_files_list_red), ncols=3, figsize=(15, 5*len(anat_files_list_red)))
#fig, axes = plt.subplots(nrows=len(anat_files_list_red), ncols=1, figsize=(10, 5*len(anat_files_list_red)))

for i, (file, matched_array) in enumerate(zip(anat_files_list_red, matched)):
    image_data = nib.load(file).get_fdata()

    axes[i, 0].set_title(f"{op.basename(file)}")

    #img_percentile = np.percentile(image_data.flatten(), 99, method="nearest")+20
    img_percentile = 256

    nbins=(image_data.max()/2).astype(int)

    axes[i, 0].hist(image_data.flatten(), bins=nbins, range=(0, img_percentile), color="tab:blue")
    axes[i, 0].hist(matched_array.flatten(), bins=nbins, range=(0, img_percentile), color="tab:orange", alpha=.6)
    axes[i, 0].axis([0, 256, 0, 60000])
    
    axes[i, 1].imshow(image_data[..., 100], cmap="binary_r", vmin=0, vmax=256)
    axes[i, 2].imshow(matched_array[..., 100], cmap="binary_r", vmin=0, vmax=256)

In [ ]:
len(matched)

In [ ]:
anat_files_list_red = anat_files_list#[:2]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
#fig, axes = plt.subplots(nrows=len(anat_files_list_red), ncols=1, figsize=(10, 5*len(anat_files_list_red)))

img_percentile = np.percentile(first_data.flatten(), 99, method="nearest")+20
nbins=(first_data.max()/2).astype(int)

colors = plt.get_cmap("Spectral", len(anat_files_list_red)+2)

plt.hist(first_data.flatten(), bins=nbins, range=(10, img_percentile), color="tab:blue", alpha=.8)

for i, matched_img in enumerate(matched):
    plt.hist(matched_img.flatten(), bins=nbins, histtype="step", range=(0, 256), alpha=.6,
             color=colors(i), linewidth=2, label=op.basename(file))
    
plt.axis([0, 256, 0, 60000])
fig.legend(loc="upper right", bbox_to_anchor=(1.2, 0.9))